In [1]:
import itertools
from treys import Card
from treys import Evaluator
from treys import Deck

In [2]:
def lget(l, index, default=None):
    try:
        return l[index]
    except IndexError:
        return default

In [3]:
def typeOfCard(card, cardsDesired):
    if not cardsDesired:
        return True
    
    return card in cardsDesired

In [101]:
%reload_ext autoreload
%autoreload 2
from strategies import firstNOfAKind, firstFlush

# match keywords with parameters for function (N, additionalPair)
validKeyWords = {'firstFive': (1,0), 'firstPair': (2,0), 'firstTwoPair': (2,1),
                 'firstTrips': (3,0), 'firstFullHouse': (3, 1), 'firstQuads': (4,0),
                 'firstFlush': None}

def simulatePokerSolitaire(cardsDesired, keyWord = None):
    playerCards, communityCards, deck = [], [], Deck()

    if keyWord is None:
        # run with cardsDesired
        while len(playerCards) < 5 or len(communityCards) < 5:
            card = deck.draw(1)
            if len(playerCards) == 5:
                communityCards.append(card)
            elif typeOfCard(Card.print_pretty_card(card), Card.print_pretty_cards(cardsDesired)):
                playerCards.append(card)
            else:
                communityCards.append(card)
    elif keyWord not in validKeyWords:
        raise SyntaxError("Invalid keyword")
    elif keyWord == 'firstFlush':
        playerCards, communityCards = firstFlush(deck)
    else:
        N, additionalPair = validKeyWords[keyWord]
        keyWord = 'firstNOfAKind' + '(deck, {}, {})'.format(N, additionalPair)
        playerCards, communityCards = eval(keyWord)

    return playerCards, communityCards

In [21]:
def findBestCombination(totalCombinations, highestScore, evaluator):
    for combination in totalCombinations:
        if evaluator.evaluate(list(combination), []) == highestScore:
            return list(combination)
    
    return None

In [22]:
def evaluateBetterHand(playerCards, communityCards):
    evaluator = Evaluator()
    playerScore = evaluator.evaluate(playerCards, [])
    totalCombinations = [combinationCommunityCards for combinationCommunityCards in itertools.combinations(communityCards, 5)]
    highestScore = min(evaluator.evaluate(list(combination), []) for combination in totalCombinations)
    bestCommunityCombination = findBestCombination(totalCombinations, highestScore, evaluator)
    
    print(playerScore, highestScore)
    return highestScore >= playerScore, bestCommunityCombination

In [98]:
# note CHAR_RANK_TO_INT_RANK function doesn't exist

def playPoker(enterHand, keyWord = None):
    numericCardHand = [((CHAR_RANK_TO_INT_RANK[lget(card, 0)] << 8) + 
                        (CHAR_SUIT_TO_INT_SUIT[lget(card, 1)] << 12)) for card in enterHand]
    playerCards, communityCards = simulatePokerSolitaire(numericCardHand, keyWord)
    playerWin, bestCommunityCombination = evaluateBetterHand(playerCards, communityCards)
    
    return playerWin, Card.print_pretty_cards(playerCards), Card.print_pretty_cards(communityCards), Card.print_pretty_cards(bestCommunityCombination)

In [43]:
enterHand = ['4d', '3s', 'As', '2d', '5c']


1

In [113]:
print('First Five Cards')
print(playPoker([], "firstFive"))

print('\nFirst Pair')
print(playPoker([], "firstPair"))

print('\nFirst Two Pair')
print(playPoker([], "firstTwoPair"))

print('\nFirst Trips')
print(playPoker([], "firstTrips"))

print('\nFirst Full House')
print(playPoker([], "firstFullHouse"))

print('\nFirst Quads')
print(playPoker([], "firstQuads"))

print('\nFirst Flush')
print(playPoker([], "firstFlush"))


First Five Cards
5013 6499
(True, ' [7♣],[J♣],[3♦],[7♠],[9♥] ', ' [8♠],[6♥],[A♣],[9♠],[J♦] ', ' [8♠],[6♥],[A♣],[9♠],[J♦] ')

First Pair
4958 5452
(True, ' [7♦],[3♥],[7♥],[K♥],[6♣] ', ' [5♠],[9♦],[4♦],[5♣],[J♣] ', ' [5♠],[9♦],[4♦],[5♣],[J♣] ')

First Two Pair
3039 5216
(True, ' [7♦],[7♣],[2♣],[9♥],[9♣] ', ' [Q♦],[4♥],[5♥],[6♥],[6♠] ', ' [Q♦],[4♥],[5♥],[6♥],[6♠] ')

First Trips
1737 230
(False, ' [3♦],[5♥],[K♦],[K♥],[K♣] ', ' [2♠],[8♠],[4♥],[9♣],[J♦],[4♦],[9♦],[J♥],[9♥],[2♥],[3♣],[3♥],[2♣],[Q♣] ', ' [9♣],[J♦],[9♦],[J♥],[9♥] ')

First Full House
178 182
(True, ' [2♦],[A♣],[A♠],[2♠],[A♦] ', ' [8♥],[T♠],[7♣],[T♥],[7♦],[9♦],[K♠],[5♦],[5♥],[K♦],[J♦],[K♥],[9♣],[8♠],[3♥],[9♠] ', ' [T♠],[T♥],[K♠],[K♦],[K♥] ')

First Quads
46 4
(False, ' [Q♣],[2♠],[Q♦],[Q♠],[Q♥] ', ' [8♥],[8♣],[7♦],[3♥],[7♥],[6♦],[A♦],[3♦],[9♦],[8♠],[6♣],[5♥],[A♥],[4♣],[8♦],[4♥],[2♥],[K♦],[T♦],[9♥],[J♥],[7♣],[5♠],[J♠],[4♦],[7♠],[T♥],[J♦],[3♠],[J♣],[9♠],[4♠],[5♦],[2♦],[K♥],[9♣],[6♠],[A♣],[6♥] ', ' [8♥],[7♥],[9♥],[J♥],[T♥] ')

Firs